# Loading packages

In [457]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import seaborn as sns
from google.colab import files
!pip install xlrd
import xlrd
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers, regularizers

# Titanic dataset

In [0]:
data = files.urllib.request.urlretrieve('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls', filename=None)[0]
df = pd.read_excel(data)

## Data preprocessing

Firstly, let's get rid of features that have too many nans, as was decided in EDA.

In [0]:
df.drop(columns=['cabin', 'boat', 'body', 'home.dest', 'name'], inplace=True)

In [0]:
for column in df.columns:
  if (df[column].dtypes in ['float64', 'int64']) and sum(df[column].isnull()):
    df[column].fillna(df[column].median(), inplace=True)  
  if (df[column].dtypes == 'object') and sum(df[column].isnull()):
    df[column].fillna(df[column].mode()[0], inplace=True)

In [370]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
pclass      1309 non-null int64
survived    1309 non-null int64
sex         1309 non-null object
age         1309 non-null float64
sibsp       1309 non-null int64
parch       1309 non-null int64
ticket      1309 non-null object
fare        1309 non-null float64
embarked    1309 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 92.1+ KB


In [478]:
df.ticket = df.ticket.str.replace(' \d+', '').str.replace('[0-9][0-9]+', '').replace([np.nan, ''], 'simple')
df.ticket.unique() # I will drop this feature due to high range of categories

array(['simple', 'PC', 'W.E.P.', 'WE/P', 'F.C.', 'P/PP', 'C.A.',
       'C.A./SOTON', 'S.O.P.', 'F.C.C.', 'SC/AH', 'W./C.', 'S.O.C.',
       'SC/PARIS', 'S.W./PP', 'W/C', 'SCO/W', 'SO/C', 'SC/AH Basle',
       'SC/Paris', 'SC', 'S.O./P.P.', 'S.C./PARIS', 'SC/A.3', 'SW/PP',
       'CA', 'SOTON/O2', 'C', 'SOTON/O.Q.', 'A/4', 'PP', 'A/5', 'A./5.',
       'A/5.', 'SOTON/OQ', 'S.P.', 'SC/A4', 'AQ/3.', 'STON/O2.', 'A..',
       'LINE', 'STON/O.', 'AQ/4', 'A/S', 'A.5.', 'A4.', 'A/4.', 'Fa',
       'S.C./A.4.', 'LP', 'CA.', 'STON/OQ.'], dtype=object)

In [0]:
df.drop(columns='ticket', inplace=True)

In [0]:
binarizer = LabelBinarizer()
df.sex = binarizer.fit_transform(df.sex)
df = pd.get_dummies(df)

In [0]:
scaler = StandardScaler()
df[['age', 'fare']] = scaler.fit_transform(df[['age', 'fare']])

In [43]:
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,-0.039005,0,0,3.442584,0,0,1
1,1,1,1,-2.215952,1,2,2.286639,0,0,1
2,1,0,0,-2.131977,1,2,2.286639,0,0,1
3,1,0,1,0.038512,1,2,2.286639,0,0,1
4,1,0,0,-0.349075,1,2,2.286639,0,0,1


## Train-test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='survived'), df.survived, test_size=0.3, random_state=42)

## Neural Network building

### TensorFlow

In [483]:
n_samples = X_train.shape[0]
print( X_train.shape )
print( y_train.shape )

(916, 9)
(916,)


In [0]:
learning_rate = 0.03
n_epochs = 10

X = tf.placeholder(tf.float32, [None, 9])
y = tf.placeholder(tf.float32, [None, 1])
weights = tf.Variable(tf.random_normal([9, 1], 0.0, 0.01, tf.float32))
bias = tf.Variable(tf.zeros([1, 1]))
logits = tf.add(tf.matmul(X, weights), bias)
loss = tf.losses.sigmoid_cross_entropy(y, logits)
      #tf.losses.hinge_loss(logits=logits, labels=y)
      #tf.reduce_mean(tf.square(1-logits)) 
      #tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
predictions = tf.rint(tf.nn.sigmoid(logits))
accuracy = tf.reduce_sum(tf.cast(tf.equal(predictions, y), tf.float32))

In [494]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for i in range(n_epochs):
      total_loss=0
      for j in range(n_samples): 
        _, l = sess.run([optimizer, loss], feed_dict={X: X_train.values[j].reshape((-1, 9)), y:y_train.values[j].reshape(-1, 1)})
      total_loss += l
      pred = sess.run(predictions, {X: X_test})
      score = sess.run(accuracy, {predictions: pred, y: y_test.values.reshape(-1, 1)})
      print('Epoch {0}: {1}, Accuracy {0}: {2}'.format(i, total_loss/n_samples, score/len(pred)))
    sess.close()

Epoch 0: 0.0009383469429599145, Accuracy 0: 0.7760814249363868
Epoch 1: 0.0011231611649542396, Accuracy 1: 0.7938931297709924
Epoch 2: 0.0012037562751353568, Accuracy 2: 0.7913486005089059
Epoch 3: 0.0012390707257533178, Accuracy 3: 0.7964376590330788
Epoch 4: 0.0012544319358975607, Accuracy 4: 0.7964376590330788
Epoch 5: 0.0012607163216869905, Accuracy 5: 0.8015267175572519
Epoch 6: 0.0012627625309223692, Accuracy 6: 0.8015267175572519
Epoch 7: 0.0012628210944379782, Accuracy 7: 0.8040712468193384
Epoch 8: 0.001261985718423102, Accuracy 8: 0.8040712468193384
Epoch 9: 0.0012608082013359236, Accuracy 9: 0.8040712468193384


### Keras

In [387]:
adam = optimizers.Adam(lr=0.03)
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(9,)))
model.compile(adam, 'binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=1, epochs=10,
          validation_data = (X_test, y_test))

Train on 916 samples, validate on 393 samples
Epoch 1/10
916/916 [==============================] - 3s 3ms/step - loss: 0.5322 - acc: 0.7456 - val_loss: 0.4950 - val_acc: 0.7913
Epoch 2/10
916/916 [==============================] - 1s 2ms/step - loss: 0.5061 - acc: 0.7664 - val_loss: 0.4606 - val_acc: 0.7964
Epoch 3/10
916/916 [==============================] - 1s 2ms/step - loss: 0.4874 - acc: 0.7948 - val_loss: 0.5674 - val_acc: 0.7455
Epoch 4/10
916/916 [==============================] - 1s 2ms/step - loss: 0.4909 - acc: 0.7838 - val_loss: 0.4612 - val_acc: 0.8015
Epoch 5/10
916/916 [==============================] - 1s 2ms/step - loss: 0.4951 - acc: 0.7784 - val_loss: 0.4998 - val_acc: 0.7735
Epoch 6/10
916/916 [==============================] - 1s 2ms/step - loss: 0.4939 - acc: 0.7817 - val_loss: 0.4785 - val_acc: 0.7812
Epoch 7/10
916/916 [==============================] - 1s 2ms/step - loss: 0.4978 - acc: 0.7817 - val_loss: 0.4970 - val_acc: 0.7913
Epoch 8/10
916/916 [==========

# Thyroid dataset

In [0]:
data2 = files.urllib.request.urlretrieve('https://www.openml.org/data/get_csv/57/dataset_57_hypothyroid.arff', filename=None)[0]
df2 = pd.read_csv(data2)

## Data preprocessing

In [0]:
df2.replace('?', np.nan, inplace=True)
df2.drop(columns=['TBG', 'TBG_measured'], inplace=True)

In [0]:
for i in df2:
  try:
    df2[i] = df2[i].astype(float)
  except:
    continue

In [391]:
df2.age[1364] = df2.age.median()
df2.age.fillna(df2.age.median(), inplace=True)
df2.sex.fillna(df2.sex.mode().iloc[0], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
for column in df2.columns:
  if sum(df2[column].isnull()):
    df2[column].fillna(df2[column].median(), inplace=True)

In [161]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 28 columns):
age                          3772 non-null float64
sex                          3772 non-null object
on_thyroxine                 3772 non-null object
query_on_thyroxine           3772 non-null object
on_antithyroid_medication    3772 non-null object
sick                         3772 non-null object
pregnant                     3772 non-null object
thyroid_surgery              3772 non-null object
I131_treatment               3772 non-null object
query_hypothyroid            3772 non-null object
query_hyperthyroid           3772 non-null object
lithium                      3772 non-null object
goitre                       3772 non-null object
tumor                        3772 non-null object
hypopituitary                3772 non-null object
psych                        3772 non-null object
TSH_measured                 3772 non-null object
TSH                          3772 non-null

In [0]:
minority = df2[df2.Class != 'negative']

In [0]:
for i in range(5):
  df2 = df2.append(minority) #oversampling (as data is imbalanced)

In [0]:
scale = StandardScaler()
df2[['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI']] = scale.fit_transform(df2[['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI']])

In [0]:
binarizer = LabelBinarizer()
encoder = LabelEncoder()
df2.Class = encoder.fit_transform(df2.Class)
for column in df2.columns:
  if len(df2[column].unique()) == 2:
    #print( column )
    df2[column] = binarizer.fit_transform(df2[column])
 

In [0]:
df2 = pd.get_dummies(df2, columns=['referral_source'])

In [356]:
df2.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,T4U_measured,T4U,FTI_measured,FTI,Class,referral_source_STMW,referral_source_SVHC,referral_source_SVHD,referral_source_SVI,referral_source_other
0,-0.557695,0,0,0,0,0,0,0,0,0,...,1,0.772381,1,0.235853,1,0,1,0,0,0
1,-1.504881,0,0,0,0,0,0,0,0,0,...,0,-0.101484,0,0.179295,1,0,0,0,0,1
2,-0.294588,1,0,0,0,0,0,0,0,0,...,1,-0.483800,1,0.546927,1,0,0,0,0,1
3,0.968328,0,1,0,0,0,0,0,0,0,...,0,-0.101484,0,0.179295,1,0,0,0,0,1
4,0.968328,0,0,0,0,0,0,0,0,0,...,1,-0.702266,1,-0.867043,1,0,0,0,1,0


In [0]:
df2.info()

## Train-test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df2.drop(columns='Class'), df2.Class, test_size=0.3, stratify=df2.Class, random_state=42)

In [0]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## Neural Network building

### TensorFlow

In [400]:
n_samples = X_train.shape[0]
print( X_train.shape )
print( y_train.shape )

(3658, 31)
(3658, 4)


In [0]:
learning_rate = 0.01
n_epochs = 10

X = tf.placeholder(tf.float32, [None, 31])
y = tf.placeholder(tf.float32, [None, 4])
weights = tf.Variable(tf.random_normal([31, 4], 0.0, 0.01, tf.float32))
bias = tf.Variable(tf.zeros([1, 4]))
logits = tf.add(tf.matmul(X, weights), bias)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
predictions = tf.nn.softmax(logits)
correct_preds = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1)), tf.float32))
#accuracy = tf.reduce_sum(correct_preds)

In [402]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for i in range(n_epochs):
      total_loss=0
      for j in range(n_samples): 
        _, l = sess.run([optimizer, loss], feed_dict={X: X_train.values[j].reshape(-1, 31), y:y_train.values[j].reshape(-1, 4)})
        total_loss += l
      pred = sess.run(predictions, {X: X_test})
      score = sess.run(correct_preds, {predictions: pred, y:y_test})
      #score = sess.run(accuracy, {correct_preds: correct_preds})
      print('Epoch {0}: {1}, Accuracy {0}: {2}, ROC_AUC {0}: {3}'.format(i, total_loss/n_samples, score/len(pred), roc_auc_score(y_test, pred)))
    sess.close()

Epoch 0: 0.38362111854131975, Accuracy 0: 0.9133205863607393, ROC_AUC 0: 0.9865839822874807
Epoch 1: 0.25368487406524703, Accuracy 1: 0.940089228808158, ROC_AUC 1: 0.9925593357101373
Epoch 2: 0.21734792016039753, Accuracy 2: 0.9483747609942639, ROC_AUC 2: 0.994473364749946
Epoch 3: 0.19944510404548293, Accuracy 3: 0.9509241555130656, ROC_AUC 3: 0.9956914860722775
Epoch 4: 0.1890492380823574, Accuracy 4: 0.9547482472912683, ROC_AUC 4: 0.9961844172468804
Epoch 5: 0.18239055238767157, Accuracy 5: 0.9553855959209687, ROC_AUC 5: 0.9964696056771702
Epoch 6: 0.1777759615643913, Accuracy 6: 0.9592096876991715, ROC_AUC 6: 0.9966856331328156
Epoch 7: 0.1743981443829522, Accuracy 7: 0.9649458253664754, ROC_AUC 7: 0.9968163119970127
Epoch 8: 0.17185349753179416, Accuracy 8: 0.9655831739961759, ROC_AUC 8: 0.9970055580238915
Epoch 9: 0.1699310950272928, Accuracy 9: 0.9681325685149776, ROC_AUC 9: 0.997114823327664


### Keras

In [403]:
adam = optimizers.Adam(lr=0.03)
model = Sequential()
model.add(Dense(4, activation='softmax', input_shape=(31,)))
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=1, epochs=10,
          validation_data = (X_test, y_test))

Train on 3658 samples, validate on 1569 samples
Epoch 1/10
3658/3658 [==============================] - 7s 2ms/step - loss: 0.3308 - acc: 0.8734 - val_loss: 0.2601 - val_acc: 0.9229
Epoch 2/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.2069 - acc: 0.9349 - val_loss: 0.2763 - val_acc: 0.9018
Epoch 3/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.1804 - acc: 0.9412 - val_loss: 0.1513 - val_acc: 0.9573
Epoch 4/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.1676 - acc: 0.9505 - val_loss: 0.1652 - val_acc: 0.9586
Epoch 5/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.1603 - acc: 0.9568 - val_loss: 0.2557 - val_acc: 0.9407
Epoch 6/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.1550 - acc: 0.9587 - val_loss: 0.1359 - val_acc: 0.9707
Epoch 7/10
3658/3658 [==============================] - 6s 2ms/step - loss: 0.1466 - acc: 0.9615 - val_loss: 0.1958 - val_acc: 0.9280
Epoch 8/10
365

In [404]:
roc_auc_score(y_test, model.predict(X_test))

0.996742792173108

# Boston houses dataset

In [0]:
from sklearn.datasets import load_boston

In [0]:
data = load_boston()
df_feat = pd.DataFrame(data.data, columns=data.feature_names)
df_targ = pd.DataFrame(data.target)

In [412]:
print( data.shape )
print( target.shape )

(506, 13)
(506,)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_feat, df_targ, test_size=0.3, random_state=42)

In [0]:
data = pd.DataFrame(data).fillna(data.mean())

### Simple NN

In [465]:
model = Sequential()
model.add(Dense(13, activation='relu', kernel_initializer='normal', input_shape=(13,)))
model.add(Dense(1, kernel_initializer='normal', input_shape=(13,)))
model.compile('adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train,
          batch_size=1, epochs=15,
          validation_data=(X_test, y_test))

Train on 354 samples, validate on 152 samples
Epoch 1/15
354/354 [==============================] - 3s 8ms/step - loss: 148.1496 - mean_absolute_error: 8.8285 - val_loss: 65.1548 - val_mean_absolute_error: 5.5794
Epoch 2/15
354/354 [==============================] - 1s 2ms/step - loss: 77.2678 - mean_absolute_error: 6.3950 - val_loss: 52.4835 - val_mean_absolute_error: 4.9984
Epoch 3/15
354/354 [==============================] - 1s 2ms/step - loss: 65.9015 - mean_absolute_error: 5.7971 - val_loss: 55.2473 - val_mean_absolute_error: 5.9849
Epoch 4/15
354/354 [==============================] - 1s 2ms/step - loss: 60.5702 - mean_absolute_error: 5.5111 - val_loss: 45.2853 - val_mean_absolute_error: 4.8638
Epoch 5/15
354/354 [==============================] - 1s 2ms/step - loss: 57.1050 - mean_absolute_error: 5.3516 - val_loss: 44.8524 - val_mean_absolute_error: 5.1925
Epoch 6/15
354/354 [==============================] - 1s 2ms/step - loss: 53.3671 - mean_absolute_error: 5.1471 - val_loss:

### Multi-layer NN

In [474]:
model = Sequential()
model.add(Dense(13, activation='elu', kernel_initializer='normal', activity_regularizer=regularizers.l2(0.02), input_shape=(13,)))
model.add(Dense(5, activation='elu', kernel_initializer='normal', activity_regularizer=regularizers.l2(0.02), input_shape=(13,)))
model.add(Dense(3, activation='relu', kernel_initializer='normal', activity_regularizer=regularizers.l2(0.02), input_shape=(5,)))
model.add(Dense(1, kernel_initializer='normal', activity_regularizer=regularizers.l2(0.02), input_shape=(3,)))
model.compile('adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train,
          batch_size=1, epochs=20,
          validation_data=(X_test, y_test))

Train on 354 samples, validate on 152 samples
Epoch 1/20
354/354 [==============================] - 3s 10ms/step - loss: 513.6005 - mean_absolute_error: 18.7115 - val_loss: 194.6000 - val_mean_absolute_error: 8.0576
Epoch 2/20
354/354 [==============================] - 1s 2ms/step - loss: 162.6525 - mean_absolute_error: 6.3953 - val_loss: 107.7749 - val_mean_absolute_error: 4.4111
Epoch 3/20
354/354 [==============================] - 1s 2ms/step - loss: 111.2322 - mean_absolute_error: 5.3488 - val_loss: 91.0508 - val_mean_absolute_error: 4.5260
Epoch 4/20
354/354 [==============================] - 1s 2ms/step - loss: 97.9182 - mean_absolute_error: 5.1517 - val_loss: 101.7304 - val_mean_absolute_error: 5.9129
Epoch 5/20
354/354 [==============================] - 1s 2ms/step - loss: 81.2983 - mean_absolute_error: 4.5347 - val_loss: 61.2714 - val_mean_absolute_error: 3.5840
Epoch 6/20
354/354 [==============================] - 1s 2ms/step - loss: 72.5144 - mean_absolute_error: 4.3851 - va